In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

import optuna

import pickle

from catboost import CatBoostRegressor

from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

# 1. Загрузка и обработка данных

In [2]:
df = pd.read_csv('../data/processed_data.csv')

df.head(2)

,baths,fireplace,city,sqft,beds,state,stories,target,private_pool,Year built,...,Heating_other,Cooling_central,Cooling_cooling,Cooling_other,Cooling_refrigeration,Cooling_wall,Parking_garage,Parking_no data,Parking_other,Parking_parking
0,4,1,248,2900.0,4,18,1,418.0,0,196,...,False,False,False,False,False,False,False,True,False,False
1,3,0,248,1.9,3,32,2,310.0,0,196,...,False,False,False,False,False,False,False,True,False,False


In [3]:
# определение констант
RANDOM_STATE = 42

Разделим данные на обучающую и тестовую выборки

In [4]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# 2. Baseline

Создадим простую, наивную модель, которая будет использоваться в качестве базовой для предсказания стоимости домов. Для предсказания будет взято среднее значение цены.

In [5]:
mean_target_value = y_train.mean()

naive_df = pd.DataFrame(
    {'naive_prediction': y_test.values}
)
naive_df['naive_prediction'] = mean_target_value

baseline_mape   = mean_absolute_percentage_error(y_test, naive_df['naive_prediction'])
baseline_mae    = mean_absolute_error(y_test, naive_df['naive_prediction'])

print(f'Baseline MAPE: {baseline_mape.round(2)}%')
print(f'Baseline MAE: {baseline_mae.round(2)}$')

# Сохранение модели
with open('../models/baseline_model.pkl', 'wb') as f:
    pickle.dump(naive_df, f)


Baseline MAPE: 21.05%
Baseline MAE: 182.16$


# 3. Pipeline

Проверим работу нескольких моделей со значениями по умолчанию, для сравнения их эффективности между собой и базовой моделью. В качестве моделей будем использовать:
* Linear Regression
* Random Forest
* XGBoost
* Stacking Regressor
    * estimators:
        * DecisionTreeRegressor
        * RandomForestRegressor
    * final_estimator
        * RandomForestRegressor

Для более удобной работы с моделями, будем использовать pipeline для их обучения.

In [6]:
# список моделей
models = [
    ('Linear Regression',   LinearRegression()),
    ('Random Forest',       RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)),
    ('XGBoost',             XGBRegressor(random_state=RANDOM_STATE, n_jobs=-1)),
    ('CatBoost Regressor',  CatBoostRegressor(random_state=RANDOM_STATE, silent=True)),
    ('Stacking Regressor',  StackingRegressor(
        estimators=[
            ('dt', DecisionTreeRegressor(random_state=RANDOM_STATE)),
            ('rf', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
        ],
        final_estimator=CatBoostRegressor(random_state=RANDOM_STATE, silent=True)
    ))
]

# пустой список для записи результатов
best_scores = []

# pipeline модели
for model_name, model in models:
    pipeline = Pipeline([
        ('scaler',  StandardScaler()),
        ('model',   model)
    ])

    # обучение модели
    pipeline.fit(X_train, y_train)

    # получение предсказаний на тестовой выборке
    y_pred  = pipeline.predict(X_test)

    # рассчитываем метрики
    mape    = mean_absolute_percentage_error(y_test, y_pred)
    mae     = mean_absolute_error(y_test, y_pred)

    # записываем результаты в список
    best_scores.append({
        'Model': model_name,
        'MAPE': mape.round(3),
        'MAE': mae.round(3),
    })

# добавим к спису моделей Baseline
best_scores.append({
        'Model': 'Baseline',
        'MAPE': baseline_mape.round(3),
        'MAE': baseline_mae.round(3),
    })

# создаем DataFrame из списка результатов
best_scores_df = pd.DataFrame(best_scores)

# выводим метрики и параметры лучшей модели
best_model = best_scores_df.loc[best_scores_df['MAPE'].idxmin()]
print('Best Model:', best_model['Model'])
print('MAPE:', best_model['MAPE'])
print('MAE:', best_model['MAE'])

Best Model: Stacking Regressor
MAPE: 3.997
MAE: 23.766


Сравним метрики моделей

In [7]:
best_scores_df.sort_values(by='MAPE', ignore_index=True)

,Model,MAPE,MAE
0,Stacking Regressor,3.997,23.766
1,Random Forest,4.179,23.333
2,XGBoost,6.967,38.559
3,CatBoost Regressor,7.667,37.919
4,Baseline,21.049,182.162
5,Linear Regression,24.655,168.366


In [8]:
improvement_ratio       = best_model['MAPE'] / baseline_mape
improvement_percentage  = (improvement_ratio - 1) * 100

print(f'Удалось улучшить качество предсказания по сравнению базовой моделью на {-improvement_percentage.round(2)}%')

Удалось улучшить качество предсказания по сравнению базовой моделью на 81.01%


# 4. Поиск оптимальных гиперпараметров модели

Лучший рещультат показала модель **Stacking Regressor**, выполним поиск оптимальных гиперпараметров для улучшения метрики.

In [9]:
def objective(trial):
    """
    функция для оптимизации гиперпараметров с помощью Optuna
    """
    
    dt_max_depth            = trial.suggest_int('dt_max_depth', 20, 30)
    dt_min_samples_leaf     = trial.suggest_int('dt_min_samples_leaf', 2, 8)
    dt_min_samples_split    = trial.suggest_int('dt_min_samples_split', 2, 8)

    rf_max_depth        = trial.suggest_int('rf_max_depth', 20, 30)
    rf_n_estimators     = trial.suggest_int('rf_n_estimators', 100, 200)

    cat_iterations      = trial.suggest_int('cat_iterations', 80, 120)
    cat_l2_leaf_reg     = trial.suggest_float('cat_l2_leaf_reg', 1, 5)
    cat_learning_rate   = trial.suggest_float('cat_learning_rate', 0.1, 0.8)
    cat_depth           = trial.suggest_int('cat_depth', 5, 10)

    dt = DecisionTreeRegressor(max_depth=dt_max_depth,
                               min_samples_leaf=dt_min_samples_leaf,
                               min_samples_split=dt_min_samples_split)

    rf = RandomForestRegressor(max_depth=rf_max_depth,
                               n_estimators=rf_n_estimators,
                               random_state=RANDOM_STATE, n_jobs=-1)

    cb = CatBoostRegressor(iterations=cat_iterations,
                           l2_leaf_reg=cat_l2_leaf_reg,
                           learning_rate=cat_learning_rate,
                           depth=cat_depth,
                           random_state=RANDOM_STATE, silent=True)

    # создание StackingRegressor
    stacking_regressor = StackingRegressor(estimators=[('dt', dt), ('rf', rf)],
                                           final_estimator=cb)

    # обучение StackingRegressor
    stacking_regressor.fit(X_train, y_train)

    # вычисление MAPE на валидационной выборке
    y_pred = stacking_regressor.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mape

In [10]:
# запуск оптимизации Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

# получение лучших гиперпараметров из Optuna
best_params             = study.best_params
dt_max_depth            = best_params['dt_max_depth']
dt_min_samples_leaf     = best_params['dt_min_samples_leaf']
dt_min_samples_split    = best_params['dt_min_samples_split']
rf_max_depth            = best_params['rf_max_depth']
rf_n_estimators         = best_params['rf_n_estimators']
cat_iterations          = best_params['cat_iterations']
cat_l2_leaf_reg         = best_params['cat_l2_leaf_reg']
cat_learning_rate       = best_params['cat_learning_rate']
cat_depth               = best_params['cat_depth']

# вывод лучших гиперпараметров
print('Лучшие гиперпараметры:')
print(best_params)

[I 2023-07-30 12:03:29,091] A new study created in memory with name: no-name-b5940072-b30c-495c-87ca-0a3aaecb25f3
[I 2023-07-30 12:07:05,193] Trial 0 finished with value: 3.7674780492476923 and parameters: {'dt_max_depth': 27, 'dt_min_samples_leaf': 8, 'dt_min_samples_split': 3, 'rf_max_depth': 25, 'rf_n_estimators': 174, 'cat_iterations': 96, 'cat_l2_leaf_reg': 3.731499858879426, 'cat_learning_rate': 0.3509595418014737, 'cat_depth': 10}. Best is trial 0 with value: 3.7674780492476923.
[I 2023-07-30 12:11:03,751] Trial 1 finished with value: 3.832706652758578 and parameters: {'dt_max_depth': 22, 'dt_min_samples_leaf': 4, 'dt_min_samples_split': 3, 'rf_max_depth': 28, 'rf_n_estimators': 188, 'cat_iterations': 92, 'cat_l2_leaf_reg': 4.445185142865602, 'cat_learning_rate': 0.35408727462568146, 'cat_depth': 6}. Best is trial 0 with value: 3.7674780492476923.
[I 2023-07-30 12:15:08,137] Trial 2 finished with value: 3.8069270282191154 and parameters: {'dt_max_depth': 26, 'dt_min_samples_leaf

Лучшие гиперпараметры:
{'dt_max_depth': 27, 'dt_min_samples_leaf': 8, 'dt_min_samples_split': 3, 'rf_max_depth': 25, 'rf_n_estimators': 174, 'cat_iterations': 96, 'cat_l2_leaf_reg': 3.731499858879426, 'cat_learning_rate': 0.3509595418014737, 'cat_depth': 10}


# 5 Финальная модель

Исходя из лучших параметров, создадим финальную модель StackingRegressor с лучшими гиперпараметрами

In [12]:
dt_best = DecisionTreeRegressor(
    max_depth=dt_max_depth,
    min_samples_leaf=dt_min_samples_leaf,
    min_samples_split=dt_min_samples_split)

rf_best = RandomForestRegressor(
    max_depth=rf_max_depth,
    n_estimators=rf_n_estimators,
    random_state=RANDOM_STATE, n_jobs=-1)

cb_best = CatBoostRegressor(
    iterations=cat_iterations,
    l2_leaf_reg=cat_l2_leaf_reg,
    learning_rate=cat_learning_rate,
    depth=cat_depth,
    random_state=RANDOM_STATE, silent=True)

stacking_regressor_best = StackingRegressor(
    estimators=[('dt', dt_best), ('rf', rf_best)],
    final_estimator=cb_best)

# обучение финального StackingRegressor с лучшими гиперпараметрами
stacking_regressor_best.fit(X_train, y_train)

# вычисление MAPE и MAE на тестовой выборке
y_pred_best = stacking_regressor_best.predict(X_test)
mape_best = mean_absolute_percentage_error(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)

# вывод лучших метрик оценки
print('Лучший MAPE:', mape_best.round(3))
print('Лучший MAE:', mae_best.round(3))

# сохранение лучшей модели в файл pickle
with open('../models/StackingRegressor_model.pkl', 'wb') as file:
    pickle.dump(stacking_regressor_best, file)

Лучший MAPE: 3.776
Лучший MAE: 23.929


In [13]:
improvement_ratio       = mape_best / best_model['MAPE']
improvement_percentage  = (improvement_ratio - 1) * 100

print(f'Удалось улучшить качество предсказания после поиска гиперпараметров модели на {-improvement_percentage.round(2)}%')

Удалось улучшить качество предсказания после поиска гиперпараметров модели на 5.52%


Модель готова, обучена на оптимальных параметрах и сохранена в вде pickle файла.

Готовая модель предсказывает стоимость недвижимости с погрешностью в 4%, то есть ошибается примерно на 24$. Данный показатель можно считать очень хорошим.